# Model Training

## Compute Class Weights

To handle class imbalance in Machine Learning, there are several methods.
<br><br>
One of them is adjusting the class weights. 
<br><br>
By giving higher weights to the minority class and lower weights to the majority class, we can regularize the loss function.
<br><br>
Misclassifying the minority class will result in a higher loss due to the higher weight.
<br><br>
To incorporate class weights in Tensorflow, use `scikit-learn`'s `compute_class_weight` function

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.utils import compute_class_weight

X, y = ...

# will return an array with weights for each class, e.g. [0.6, 0.6, 1.]
class_weights = compute_class_weight(
  class_weight="balanced",
  classes=np.unique(y),
  y=y
)

# to get a dictionary with {<class>:<weight>}
class_weights = dict(enumerate(class_weights))

model = tf.keras.Sequential(...)
model.compile(...)

# using class_weights in the .fit() method
model.fit(X, y, class_weight=class_weights, ...)